In [1]:
!pip install survshap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.7/535.7 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 32.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [2]:
from survshap import SurvivalModelExplainer, PredictSurvSHAP, ModelSurvSHAP
from sksurv.ensemble import RandomSurvivalForest
from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


%matplotlib inline

In [3]:
X, y = load_gbsg2()

grade_str = X.loc[:, "tgrade"].astype(object).values[:, np.newaxis]
grade_num = OrdinalEncoder(categories=[["I", "II", "III"]]).fit_transform(grade_str)

X = X.drop("tgrade", axis=1)
X = OneHotEncoder().fit_transform(X)
X.loc[:, "tgrade"] = grade_num

random_state = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)

In [4]:
model = RandomSurvivalForest(
    n_estimators=1000, min_samples_split=10, min_samples_leaf=15, n_jobs=-1, random_state=random_state
)
model.fit(X_train, y_train)

RandomSurvivalForest(min_samples_leaf=15, min_samples_split=10,
                     n_estimators=1000, n_jobs=-1, random_state=42)

In [5]:
explainer = SurvivalModelExplainer(model = model, data = X, y = y)

# compute SHAP values for a single instance
observation_A = X.iloc[[0]]
survshap_A = PredictSurvSHAP()
survshap_A.fit(explainer = explainer, new_observation = observation_A)

survshap_A.result
survshap_A.plot()

In [8]:
# compute SHAP values for a group of instances
model_survshap = ModelSurvSHAP(calculation_method="treeshap") # fast implementation for tree-based models
model_survshap.fit(explainer = explainer, new_observations = X.iloc[:10,:])

model_survshap.result
model_survshap.plot_mean_abs_shap_values()
model_survshap.plot_shap_lines_for_all_individuals(variable = "age")
extracted_survshap = model_survshap.individual_explanations[0]

100%|===================| 8979/9000 [07:16<00:01]       

In [7]:
X.columns

Index(['age', 'estrec', 'horTh=yes', 'menostat=Post', 'pnodes', 'progrec',
       'tsize', 'tgrade'],
      dtype='object')